In [ ]:
import pandas as pd
from collections import Counter
import re
import math
df = pd.read_csv("Data/2019_DBSCAN_new3.csv", encoding='utf-8-sig')

In [ ]:
df = df.sort_values(by='labels', ascending=True)
df.tail()

In [ ]:
df['labels'].max()+1

In [ ]:
## pmi 적용하지 않은 N_list에서 Counter를 적용해 상위 20개 추출 후 파일로 저장
no_pmi=[]

for i in range(df['labels'].max()+1):
    #print(i,": ====================")
    word_list=[]
    for article in df[df['labels'] == i]['N_list']:
        
       
        arr = re.sub("[\['\]]",'', article).split(',')
        for j in range(len(arr)):
            arr[j]=arr[j].strip()
    
        word_list+=arr
    
    counts=Counter(word_list).most_common(20)
    no_pmi.append(counts)




with open('no_pmi_counter.txt','w',encoding='utf-8-sig') as f:
    for i,item in enumerate(no_pmi):
        f.write("{} cluster===========\n".format(i))
        f.write(str(item))
        f.write('\n')



In [ ]:
def npmi(a, b, n, z):
    x = math.log10((n*z)/(a*b))
    y = math.log10(n/z)
    return x / y

In [ ]:
# 뉴스 기사별 PMI 체크
merge_tokens = []
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []
    merge_token2 = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        context += arr

    #2 PMI 적용
    for line in doc:
        com += list(zip(line, line[1:]))
    n = len(context)
    cnt = Counter(context)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+"_"+key[1] for key, item in result if item >= 0.91]
    if len(merge_token) > 0:
        merge_tokens+=merge_token
    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = '_'.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)
z = Counter(merge_tokens).most_common()
print(z)

In [ ]:
df['N_list'] = docs
df['tokens'] = merge_tokens
df.to_csv("./Data/2019_DBSCAN.csv", encoding="utf-8-sig", index=False)

In [ ]:
# 클러스터별 pmi 체크
cluster = {}
for i in range(df['labels'].max()+1):
    print(i,": ================")
    word_list = []
    com=[]
    for article in df[df['labels'] == i]['N_list']:
        # str -> list
        for line in article.split('], ['):
            arr = re.sub("[\[,'\]]", '', line).split(' ')
            word_list += arr
            com += list(zip(arr, arr[1:]))
    print(len(word_list))
    print(len(com))
    n = len(word_list)
    cnt = Counter(word_list)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+"_"+key[1] for key, item in result if item >= 0.9]
    # merge_token = [(key, item) for key, item in result if item >= 0.9]
    print(merge_token)
    cluster[i] = merge_token

In [ ]:
# 뉴스 기사별 N_GRAM 체크
merge_tokens = []
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        com += list(zip(arr, arr[1:]))

    #2 N_GRAM 체크
    z = Counter(com).most_common()
    for item, value in z:
        if value > 3:
            merge_token.append(item[0]+'_'+item[1])
    if len(merge_token) > 0:
        merge_tokens += merge_token

    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = '_'.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)
z = Counter(merge_tokens).most_common()
print(z)

In [ ]:
#클러스터별 N GRAM


for i in range(df['labels'].max()+1):
    docs = []
    merge_tokens = []
    print(i, "===================")
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        com = []
        context = []
        merge_token = []

        # str -> list
        for line in article.split('], ['):
            arr = re.sub("[\[,'\]]", '', line).split(' ')
            doc.append(arr)
            com += list(zip(arr, arr[1:]))

        #2 N_GRAM 체크
        z = Counter(com).most_common()
        for item, value in z:
            if value > 3:
                merge_token.append(item[0]+'_'+item[1])
        if len(merge_token) > 0:
            merge_tokens += merge_token

        # 토큰을 합치는 내용
        for token in merge_token:
            for content in doc:
                for i in range(len(content)-1):
                    cmp_str = '_'.join(content[i:i+2])
                    if cmp_str == token:
                        del content[i:i+2]
                        content.insert(i, token)
        docs.append(doc)
    z = Counter(merge_tokens).most_common()
    print(z)

In [ ]:
## pmi를 적용한 토큰 병합후, 이형동의어 사전에서 찾아 대체

import pandas as pd

df = pd.read_csv("Data/K_231_CV_2019_NE_TFIDF.csv", encoding='utf-8-sig')

tokenlist={}

with open("tokenlist.txt",'r',encoding='utf-8') as f:
    for line in f:
        token,word=line.split(',')
        tokenlist[token]=word.strip()

front=[]
end=[]

for key in tokenlist.keys():
    first,second=key.split('_')
    front.append(first)
    end.append(second)

docs=[]

for article in df['N_list']:

    doc=[]

    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        del_nums=[]
        new_arr=[]
        for i in range(len(arr)-1):
            if arr[i] in front:
                if arr[i+1]==end[front.index(arr[i])]:
                    #print(arr[i],arr[i+1])
                    del_nums.append(i)
                    del_nums.append(i+1)
                    #arr.append(arr[i]+" "+arr[i+1])
                    arr.append(tokenlist[arr[i]+"_"+arr[i+1]])

      
        for i in range(len(arr)):
            if(i not in list(set(del_nums))):
                new_arr.append(arr[i])   
        
        doc.append(new_arr)
        
        
    docs.append(doc)

#docs
print("Finished")
    

df2=df
df2['N_list']=docs
df2.to_csv("./Data/2019_kmeans_231.csv", encoding="utf-8-sig", index=False)

In [ ]:
##pmi적용된 N_list에서 Couter를 이용해 상위 20개 토큰 추출 후 파일로 저장 및 토픽으로 저장

import pandas as pd
from collections import Counter
import re

df = pd.read_csv("./Data/2019_kmeans_231.csv", encoding='utf-8-sig')

df_out=pd.DataFrame()



import pandas as pd
from collections import Counter
import re

df = pd.read_csv("./Data/2019_kmeans_231.csv", encoding='utf-8-sig')

df_out=pd.DataFrame()

words=[]

pmi_count=[]

for i in range(df['labels'].max()+1):
    word_list2=[]
    #print(i,'-----------------')
    for article in df[df['labels'] == i]['N_list']:
        arr = re.sub("[\['\]]", '', article).split(',')
        for i in range(len(arr)):
            arr[i]=arr[i].strip()
        #for line in article.split('], ['):
            #arr = re.sub("[\[,'\]]", '', line).split(' ')
        word_list2+=arr
    count1=Counter(word_list2).most_common(20)
    print(count1)
    pmi_count.append(count1)
    word = [key for key, item in count1]
    words.append(word)

with open('pmi_counter.txt','w',encoding='utf-8-sig') as f:
    for i,item in enumerate(pmi_count):
        f.write("{} cluster===========\n".format(i))
        f.write(str(item))
        f.write('\n')


df_out['topics']=words

df_out.to_csv("./Data/2019_kmeans_topics_231.csv", encoding="utf-8-sig", index=False)

In [ ]:
##클러스터 N_list에서 Counter 적용후 상위 100개 토큰 추출

from collections import Counter

df1 = pd.read_csv("./Data/북미 정상 회담_token.csv", encoding='utf-8-sig')
df2 = pd.read_csv("./Data/강원도 산불_token.csv", encoding='utf-8-sig')
df3 = pd.read_csv("./Data/남북미 정상 회담_token.csv", encoding='utf-8-sig')
df4 = pd.read_csv("./Data/헝가리 유람선 사고_token.csv", encoding='utf-8-sig')


dfs=[]
dfs.append(df1)
dfs.append(df2)
dfs.append(df3)
dfs.append(df4)

whole=[]

words_cluster=pd.DataFrame()

for item in dfs:
    words=[]
    for line in item['N_list']:
        arr=re.sub("[\[,'\]]", '', line).split(' ')
        words+=arr    
    count=Counter(words).most_common(100)
    word = [key for key, item in count]
    whole.append(word)


print(whole)
words_cluster['topics']=whole
words_cluster.to_csv("./Data/2019_cluster_topics2.csv", encoding="utf-8-sig", index=False)

In [ ]:
##클러스터 N_list에서 Counter 적영후 상위 20개를 추출

from collections import Counter
import pandas as pd
import re

df1 = pd.read_csv("./Data/북미 정상 회담_token.csv", encoding='utf-8-sig')
df2 = pd.read_csv("./Data/강원도 산불_token.csv", encoding='utf-8-sig')
df3 = pd.read_csv("./Data/남북미 정상 회담_token.csv", encoding='utf-8-sig')
df4 = pd.read_csv("./Data/헝가리 유람선 사고_token.csv", encoding='utf-8-sig')

df1=df1.sample(n=int(len(df1)*0.3))
df2=df2.sample(n=int(len(df2)*0.3))
df3=df3.sample(n=int(len(df3)*0.3))
df4=df4.sample(n=int(len(df4)*0.3))



dfs=[]
dfs.append(df1)
dfs.append(df2)
dfs.append(df3)
dfs.append(df4)

whole=[]

for item in dfs:
    words=[]
    for line in item['N_list']:
        arr=re.sub("[\[,'\]]", '', line).split(' ')
        words+=arr    
    count=Counter(words).most_common(20)
    word = [key for key, item in count]
    whole.append(word)


print(whole)

#words_cluster.to_csv("./Data/2019_cluster_topics.csv", encoding="utf-8-sig", index=False)

In [ ]:
##BIGKINDS 기사 일부만 가지고와서 cosine 유사도의 평균 구하기

from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



def get_topics(percent): 

    df1 = pd.read_csv("./Data/북미 정상 회담_token.csv", encoding='utf-8-sig')
    df2 = pd.read_csv("./Data/강원도 산불_token.csv", encoding='utf-8-sig')
    df3 = pd.read_csv("./Data/남북미 정상 회담_token.csv", encoding='utf-8-sig')
    df4 = pd.read_csv("./Data/헝가리 유람선 사고_token.csv", encoding='utf-8-sig')

    df1=df1.sample(n=int(len(df1)*percent))
    df2=df2.sample(n=int(len(df2)*percent))
    df3=df3.sample(n=int(len(df3)*percent))
    df4=df4.sample(n=int(len(df4)*percent))

    dfs=[]
    dfs.append(df1)
    dfs.append(df2)
    dfs.append(df3)
    dfs.append(df4)

    whole=[]

    for item in dfs:
        words=[]
        for line in item['N_list']:
            arr=re.sub("[\[,'\]]", '', line).split(' ')
            words+=arr    
        count=Counter(words).most_common(100)
        word = [key for key, item in count]
        whole.append(word)


    return whole
#print(whole)





#코사인 유사도 계산
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

sims=[]
for i in range(100):
    doc_nouns_list=[]

    cluster_topic = get_topics(1)[1]
    oasis_topic= get_topics(0.1)[1]

    doc_nouns_list.append(' '.join(cluster_topic))
    doc_nouns_list.append(' '.join(oasis_topic))

    #print("KNC_topic : ",doc_nouns_list[0])
    #print("Oasis_topic : ",doc_nouns_list[1])

    tfidf_vect_simple = TfidfVectorizer(min_df=1)
    feature_vect_simple = tfidf_vect_simple.fit_transform(doc_nouns_list)

    # TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환. 
    feature_vect_dense = feature_vect_simple.todense()

    #첫번째 문장과 두번째 문장의 feature vector  추출
    vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
    vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

    #첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
    similarity_simple = cos_similarity(vect1, vect2)

    print('KNC_topic, Oasis_topic Cosine 유사도: {0:.3f}'.format(similarity_simple))
    sims.append(similarity_simple)

sims=np.array(sims)
sims.mean()

In [ ]:
##BIGKINDS와 INTIMES 토픽 하나 유사도 비교

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

doc_nouns_list=[]

#코사인 유사도 계산
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

    

cluster_topic = ['헝가리', '사고', '유람선', '수색', '침몰', '한국인', '부다페스트', '시신', '강', '작업', '실종자', '구조', '인양', '확인', '현장', '정부', '선체', '현지', '이날', '수습']
bigkinds_topic=['헝가리', '사고', '유람선', '침몰', '다뉴브강', '한국인', '부다페스트', '수색', '시신', '실종자', '인양', '구조', '현지', '확인', '현장', '정부', '한국', '작업', '수습', '선체']

doc_nouns_list.append(' '.join(cluster_topic))
doc_nouns_list.append(' '.join(bigkinds_topic))

print("KNC_topic : ",doc_nouns_list[0])
print("Oasis_topic : ",doc_nouns_list[1])

tfidf_vect_simple = TfidfVectorizer(min_df=1,analyzer = 'c')
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_nouns_list)

# TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환. 
feature_vect_dense = feature_vect_simple.todense()

#첫번째 문장과 두번째 문장의 feature vector  추출
vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

#첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
similarity_simple = cos_similarity(vect1, vect2)

print('KNC_topic, Oasis_topic Cosine 유사도: {0:.3f}'.format(similarity_simple))

In [ ]:
##BIGKINDS와 INTIMES 전체 토픽 유사도 비교

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import re

#코사인 유사도 계산
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity



df = pd.read_csv("Data/2019_cluster_topics3.csv", encoding='utf-8-sig')

intime_topics=[]

for line in df['topics']:
    arr=re.sub("[\[,'\]]", '', line).split(' ')
    intime_topics.append(arr)

sims=[]

for i,item in enumerate(intime_topics):
    cluster_topic = item
    bigkinds_topic=['헝가리', '사고', '유람선', '침몰', '다뉴브강', '한국인', '부다페스트', '수색', '시신', '실종자', '인양', '구조', '현지', '확인', '현장', '정부', '한국', '작업', '수습', '선체']

    doc_nouns_list=[]

    doc_nouns_list.append(' '.join(cluster_topic))
    doc_nouns_list.append(' '.join(bigkinds_topic))

    #print("KNC_topic : ",doc_nouns_list[0])
    #print("Oasis_topic : ",doc_nouns_list[1])

    tfidf_vect_simple = TfidfVectorizer(min_df=1)
    feature_vect_simple = tfidf_vect_simple.fit_transform(doc_nouns_list)

    # TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환. 
    feature_vect_dense = feature_vect_simple.todense()

    #첫번째 문장과 두번째 문장의 feature vector  추출
    vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
    vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

    #첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
    similarity_simple = cos_similarity(vect1, vect2)

    print('{0}: KNC_topic, Oasis_topic Cosine 유사도: {1:.3f}'.format(i,similarity_simple))
    sims.append(similarity_simple)
    if(similarity_simple>0.5):
        print("KNC_topic : ",item)

sims=np.array(sims)
sims.mean()
